In [1]:
import pandas as pd
from trader import Trader
from datamodel import *
import matplotlib.pyplot as plt

In [2]:

data = pd.read_csv("8cf4dead-dd7a-4882-8510-bbce815a8c28.csv",sep = ";")

In [69]:
def match_bid(order: Order, order_depth: OrderDepth,timestamp: Time):
    if order.quantity > 0:
        for ask, quantity in order_depth.sell_orders.items():    # Since we want to buy we match against the sell orders
            if ask >= order.price:
                trade=Trade(order.symbol, order.price, -quantity,"","",timestamp)
                return trade

def match_ask(order: Order, order_depth: OrderDepth,timestamp: Time):

    if order.quantity < 0:
        for bid, quantity in order_depth.buy_orders.items():    # Since we want to sell we match against the buy orders
            if bid <= order.price:
                trade=Trade(order.symbol, order.price, quantity,"","",timestamp)
                return trade

In [76]:
def match_bid(order: Order, order_depth: OrderDepth,timestamp: Time):
    if order.quantity > 0:
        for bid, quantity in order_depth.buy_orders.items():
            if bid <= order.price:
                trade=Trade(order.symbol, order.price, quantity,"","",timestamp)
                return trade
def match_ask(order: Order, order_depth: OrderDepth,timestamp: Time):
    if order.quantity < 0:
        for ask, quantity in order_depth.sell_orders.items():
            if ask >= order.price:
                trade=Trade(order.symbol, order.price, -quantity,"","",timestamp)
                return trade
#own_trades = [symbol, List[Trade(None,None,None)]]
#market_trades = [symbol, List[Trade(None, None, None)]] #I think we will not be interested in this but we need to initialize it

pos: Position = 0
own_trades = []
market_trades = []
max_holdings = {"RAINFOREST_RESIN": 50, "KELP": 50}
pnl = {product: 0 for product in max_holdings.keys()} #Will need to be updated because Im just taking the max_holdings keys
for index,row in data.iterrows():
    timestamp: Time = row["timestamp"]
    product: Product = row["product"]
    symbol: Symbol = Symbol(product)
    traderData = "Dragonera"
    bids: Dict = {row[f"bid_price_{i}"]: row[f"bid_volume_{i}"] for i in range(1, 4) if pd.notna(row[f'bid_price_{i}'])}
    asks: Dict = {row[f"ask_price_{i}"]: row[f"ask_volume_{i}"] for i in range(1, 4) if pd.notna(row[f'ask_price_{i}'])}
    order_depth = OrderDepth()
    order_depth.buy_orders = bids
    order_depth.sell_orders = asks
    order_depths = {symbol: order_depth}
    position: Dict= {product:  pos}
    listings: Dict[Symbol, Listing] = {symbol: Listing(symbol=symbol,product=product,denomination=product)}  # Assuming fair price is in the data
    best_bid = max(bids.keys(), default=None)
    best_ask = min(asks.keys(), default=None)
    mid_price = (best_bid + best_ask) / 2 if best_bid is not None and best_ask is not None else None
    transportfee = 0
    exporttarif = 0
    importtarif = 0
    sugarprice = 0
    sunlightindex = 0
    plain_value_obss: Dict[Product,ObservationValue] = {product:mid_price}
    conv_obs = ConversionObservation(bidPrice=best_bid,askPrice=best_ask,
                                        transportFees=transportfee,
                                        exportTariff=exporttarif,
                                        importTariff=importtarif,
                                        sugarPrice=sugarprice,
                                        sunlightIndex=sunlightindex)
    conversion_obss: Dict[Product, ConversionObservation] = {product: conv_obs}
    observation: Observation = Observation(conversionObservations=conversion_obss,
                                           plainValueObservations=plain_value_obss)
    state = TradingState(traderData= traderData,
                         timestamp=timestamp,
                         order_depths=order_depths,
                         own_trades=own_trades,
                         market_trades=market_trades,
                         position=position,
                         observations= observation,
                         listings=listings)
    trader = Trader()
    orders= trader.run(state)[0]
    # print(orders)
    #Once we get the orders we need to see if the order is a trade or not
    for symbol, order in orders.items():
        if order and len(order) > 0:
            order = order[0]
            #The available to trade[0] is the positive one and [1] is the negative one (short position)
            available_to_trade = [max_holdings[product] -position[product],-max_holdings[product]-position[product]]
            #Trade Matching
            if order.quantity > 0:
                if order.quantity > available_to_trade[0]:
                    break #The orders are cancelled because the quantity is too high
                else:
                    trade = match_bid(order, order_depths[order.symbol],timestamp)
                    #update the position
                    
                    position[product] += trade.quantity
                    #update the own_trades
                    own_trades.append(trade)
                    #update the PnL
                    pnl[trade.symbol] -= trade.quantity * trade.price
            else:
                if order.quantity < available_to_trade[1]:
                    break #The orders are cancelled because the quantity is too high
                else:
                    trade = match_ask(order, order_depths[order.symbol],timestamp)
                    #update the position
                    position[product] += trade.quantity
                    own_trades.append(trade)
                    #update the PnL
                    pnl[trade.symbol] -= trade.quantity * trade.price
            






In [77]:
positions

{'RAINFOREST_RESIN': -50, 'KELP': -50}

In [78]:
pnl

{'RAINFOREST_RESIN': 170908401, 'KELP': 51533396}